In [ ]:
# ===================================================================================
#           EDITOR DE VÍDEO PARA SPEEDRUNS - VERSÃO PARA BINDER
#
# Este script foi adaptado para funcionar em qualquer ambiente Jupyter, como o Binder,
# removendo as dependências específicas do Google Colab.
#
# v3.0 - Adaptação para Binder, novo fluxo de upload/download.
# ===================================================================================

#@title ▶️ Execute esta célula para iniciar a ferramenta

import cv2
import os
import base64
import subprocess
import json
import time
import threading
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from PIL import Image, ImageDraw, ImageFont
import webcolors
from tqdm.notebook import tqdm
import numpy as np


class VideoTimerApp:
    def __init__(self):
        self.video_capture = None
        self.video_props = {}
        self.uploaded_video_filename = "video_para_editar.mp4"
        self.input_video_path = self.uploaded_video_filename  # Usar caminho relativo
        self.load_entries = []
        self.font_map = self._list_system_fonts()
        self.is_rendering = False
        self.is_playing = False
        self.playback_thread = None
        self.widgets = {}
        self.ui_output = widgets.Output()
        display(self.ui_output)
        self._build_initial_ui()

    def _build_initial_ui(self):
        # Limpa arquivos de execuções anteriores
        for f in [self.input_video_path, "/content/video_sem_som.mp4", "/content/video_final.mp4", "video_sem_som.mp4", "video_final.mp4"]:
            if os.path.exists(f): os.remove(f)
            
        self.widgets['start_button'] = widgets.Button(description="▶️ Iniciar Editor", button_style='success', icon='check', layout=widgets.Layout(width='98%', height='50px'))
        self.widgets['status_output'] = widgets.Output()
        self.widgets['start_button'].on_click(self._start_processing)
        
        instructions_html = """
        <h2>Bem-vindo ao Editor de Timer para Speedruns!</h2>
        <div style='border: 1px solid #1E90FF; padding: 12px; margin-top: 10px; background-color: #f0f8ff;'>
        <b>Como começar:</b>
        <ol>
            <li>No menu de arquivos à esquerda, clique no ícone de <b>Upload (⬆️)</b>.</li>
            <li>Selecione o seu arquivo de vídeo.</li>
            <li><b>MUITO IMPORTANTE:</b> Após o upload, renomeie o arquivo para <code>video_para_editar.mp4</code>.<br>(Clique com o botão direito no arquivo > Renomear / Toque e segure no celular > Renomear).</li>
            <li>Quando o arquivo estiver com o nome correto, clique no botão "Iniciar Editor" abaixo.</li>
        </ol>
        </div>
        """
        initial_box = widgets.VBox([
            widgets.HTML(instructions_html),
            self.widgets['start_button'],
            self.widgets['status_output']
        ])
        with self.ui_output: clear_output(); display(initial_box)

    def _start_processing(self, b=None):
        with self.ui_output:
            clear_output(wait=True); print("Verificando arquivo de vídeo...")
            try:
                if not os.path.exists(self.input_video_path):
                   raise FileNotFoundError(f"Arquivo '{self.input_video_path}' não encontrado! Verifique se você fez o upload e o renomeou corretamente.")
                self._get_video_properties()
                print(f"✅ Vídeo '{os.path.basename(self.input_video_path)}' carregado com sucesso!")
                self._build_main_editor_ui()
            except Exception as e:
                clear_output(); display(HTML(f"<div style='color:red; border: 2px solid red; padding: 10px;'><b>ERRO:</b> {e}</div>"))
                self._build_initial_ui()
    
    # ... (O restante do código da classe continua aqui, sem alterações) ...
    # O código abaixo é exatamente o mesmo do original, pois não depende do Colab.

    def _get_video_properties(self):
        self.video_capture = cv2.VideoCapture(self.input_video_path)
        if not self.video_capture.isOpened(): raise IOError(f"Não foi possível abrir o arquivo de vídeo: {self.input_video_path}")
        self.video_props = {
            'fps': self.video_capture.get(cv2.CAP_PROP_FPS),
            'w': int(self.video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)),
            'h': int(self.video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)),
            'frames': int(self.video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
        }
        if not self.video_props['fps'] or self.video_props['frames'] == 0: raise IOError("O vídeo parece estar corrompido, ter 0 frames ou FPS inválido.")

    def _list_system_fonts(self):
        font_dirs = ["/usr/share/fonts/truetype/"]
        font_map = {}
        for dir_path in font_dirs:
            if os.path.exists(dir_path):
                for root, _, filenames in os.walk(dir_path):
                    for filename in filenames:
                        if filename.lower().endswith((".ttf", ".otf")):
                            font_name = os.path.splitext(filename)[0].replace("-", " ").title()
                            font_map[font_name] = os.path.join(root, filename)
        if not font_map: return {"Default": "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"}
        return dict(sorted(font_map.items()))

    def _format_time(self, total_seconds, format_str="M:SS.ms2"):
        if total_seconds is None: return ""
        if abs(total_seconds) < 0.001: total_seconds = 0.0
        sign = "-" if total_seconds < 0 else ""
        total_seconds = abs(total_seconds)
        total_milliseconds = int(round(total_seconds * 1000))
        h, rem_h = divmod(total_milliseconds, 3600000)
        m_in_h, rem_m = divmod(rem_h, 60000)
        s_in_m_h, ms_in_s_h = divmod(rem_m, 1000)
        total_m, rem_total_m = divmod(total_milliseconds, 60000)
        s_in_total_m, ms_in_s_m = divmod(rem_total_m, 1000)
        formats = {
            "M:SS": f"{sign}{total_m}:{s_in_total_m:02d}",
            "M:SS.ms2": f"{sign}{total_m}:{s_in_total_m:02d}.{ms_in_s_m//10:02d}",
            "M:SS.ms3": f"{sign}{total_m}:{s_in_total_m:02d}.{ms_in_s_m:03d}",
            "MM:SS.ms3": f"{sign}{total_m:02d}:{s_in_total_m:02d}.{ms_in_s_m:03d}",
            "H:MM:SS": f"{sign}{h}:{m_in_h:02d}:{s_in_m_h:02d}",
            "H:MM:SS.ms2": f"{sign}{h:02d}:{m_in_h:02d}:{s_in_m_h:02d}.{ms_in_s_h//10:02d}"
        }
        return formats.get(format_str, formats["M:SS.ms2"])

    def _parse_time_to_seconds(self, time_str):
        if not isinstance(time_str, str) or not time_str.strip(): return 0.0
        try:
            sign = -1 if time_str.startswith('-') else 1
            time_str = time_str.lstrip('-')
            if ':' in time_str:
                parts = time_str.split(':')
                if len(parts) == 3: h, m, s_ms = map(float, parts); return sign * (h * 3600 + m * 60 + s_ms)
                elif len(parts) == 2: m, s_ms = map(float, parts); return sign * (m * 60 + s_ms)
            parts = [float(p) for p in time_str.split('.')] 
            if len(parts) == 2: s,ms = parts; return sign * (s + ms / (1000.0 if ms > 99 else 100.0))
            if len(parts) == 1: s = parts[0]; return sign * s
            return 0.0
        except (ValueError, IndexError): return 0.0

    def _parse_time_to_frames(self, time_str, fps): return int(round(self._parse_time_to_seconds(time_str) * fps))

    def _calculate_ingame_seconds(self, current_frame, start_run_frame, load_frames, fps):
        if current_frame < start_run_frame: return 0.0
        time_elapsed = (current_frame - start_run_frame) / fps
        load_duration = 0
        for load_start, load_end in load_frames:
            eff_start = max(start_run_frame, load_start)
            eff_end = min(current_frame, load_end)
            if eff_end > eff_start: load_duration += (eff_end - eff_start) / fps
        return max(0, time_elapsed - load_duration)

    def _build_main_navigation_panel(self):
        btn_layout = widgets.Layout(width='auto', flex='1 1 0%', margin='2px')
        def on_nav_click(diff, unit):
            if self.is_playing: self._toggle_playback()
            current_frame = self.widgets['preview_slider'].value
            if unit == 'f': step = diff
            elif unit == 's': step = int(diff * self.video_props['fps'])
            elif unit == 'm': step = int(diff * 60 * self.video_props['fps'])
            self.widgets['preview_slider'].value = max(0, min(self.video_props['frames'] - 1, current_frame + step))
        def on_set_click(widget_key):
            current_time = self.widgets['preview_slider'].value / self.video_props['fps']
            self.widgets[widget_key].value = self._format_time(current_time, self.widgets['input_time_format'].value)
        def go_to_marker(widget_key):
            if self.is_playing: self._toggle_playback()
            self.widgets['preview_slider'].value = self._parse_time_to_frames(self.widgets[widget_key].value, self.video_props['fps'])
        set_start_btn = widgets.Button(description="SET Início", tooltip="Definir o frame atual como início da run", layout=btn_layout)
        set_end_btn = widgets.Button(description="SET Fim", tooltip="Definir o frame atual como fim da run", layout=btn_layout)
        self.widgets['play_pause_button'] = widgets.Button(icon="play", layout=widgets.Layout(width='50px', margin='2px'))
        go_to_start_btn = widgets.Button(description="▶| Início", tooltip="Ir para o frame de início da run", layout=btn_layout)
        go_to_end_btn = widgets.Button(description="Fim |◀", tooltip="Ir para o frame de fim da run", layout=btn_layout)
        go_to_start_btn.on_click(lambda b: go_to_marker('start_run_widget')); go_to_end_btn.on_click(lambda b: go_to_marker('end_run_widget'))
        set_start_btn.on_click(lambda b: on_set_click('start_run_widget')); set_end_btn.on_click(lambda b: on_set_click('end_run_widget'))
        self.widgets['play_pause_button'].on_click(lambda b: self._toggle_playback())
        top_row = widgets.HBox([set_start_btn, go_to_start_btn, self.widgets['play_pause_button'], go_to_end_btn, set_end_btn], layout=widgets.Layout(flex_flow='row wrap', justify_content='center'))
        nav_rows = []
        for unit, unit_label in [('m', 'Min'), ('s', 'Sec'), ('f', 'Frame')]:
            buttons = [widgets.Button(description=f"{val}{unit_label}" if val < 0 else f"+{val}{unit_label}", layout=btn_layout) for val in [-10, -5, -1, 1, 5, 10]]
            for btn, val in zip(buttons, [-10, -5, -1, 1, 5, 10]): btn.on_click(lambda b, v=val, u=unit: on_nav_click(v, u))
            nav_rows.append(widgets.HBox(buttons, layout=widgets.Layout(flex_flow='row wrap', justify_content='center')))
        return widgets.VBox([top_row] + nav_rows, layout=widgets.Layout(margin='10px 0'))

    def _toggle_playback(self):
        self.is_playing = not self.is_playing; self.widgets['play_pause_button'].icon = 'pause' if self.is_playing else 'play'
        if self.is_playing: self.playback_thread = threading.Thread(target=self._playback_thread_func); self.playback_thread.start()

    def _playback_thread_func(self):
        while self.is_playing:
            slider = self.widgets['preview_slider']
            if slider.value >= self.video_props['frames'] - 1: break
            slider.value += 1; time.sleep(1 / self.video_props['fps'])
        self.is_playing = False
        if 'play_pause_button' in self.widgets: self.widgets['play_pause_button'].icon = 'play'

    def _build_main_editor_ui(self):
        style = {'description_width': 'initial'}
        all_time_formats = ["M:SS.ms2", "H:MM:SS.ms2", "M:SS", "H:MM:SS", "M:SS.ms3", "MM:SS.ms3"]
        self.widgets = {}
        self.widgets['render_output'] = widgets.Output()
        self.widgets['preview_slider'] = widgets.IntSlider(min=0, max=self.video_props["frames"]-1, step=1, layout=widgets.Layout(width="100%"), continuous_update=False)
        self.widgets['preview_time_label'] = widgets.Label()
        self.widgets['preview_out'] = widgets.Output()
        self.widgets['start_run_widget'] = widgets.Text(value="0:00.00", description="Início da Run:", style=style)
        self.widgets['end_run_widget'] = widgets.Text(value=self._format_time(self.video_props['frames']/self.video_props['fps']), description="Fim da Run:", style=style)
        self.widgets['start_run_preview'] = widgets.Output(); self.widgets['end_run_preview'] = widgets.Output()
        self.widgets['load_container'] = widgets.VBox()
        self.widgets['add_load_btn'] = widgets.Button(description="+ Add Load", button_style='success', icon='pause-circle')
        default_font = "Roboto" if "Roboto" in self.font_map else next(iter(self.font_map.keys()))
        self.widgets['font_widget'] = widgets.Dropdown(options=self.font_map.keys(), value=default_font, description="Fonte:", style=style)
        self.widgets['timer_font_size'] = widgets.IntSlider(value=80, min=20, max=200, description="Tamanho:", style=style)
        self.widgets['timer_color_widget'] = widgets.ColorPicker(concise=True, description="Cor:", value="#FFFFFF", style=style)
        alignment_options = { "Inferior Direito": "rs", "Inferior Central": "ms", "Inferior Esquerdo": "ls", "Superior Direito": "rt", "Superior Central": "mt", "Superior Esquerdo": "lt", "Centro": "mm" }
        self.widgets['timer_alignment'] = widgets.Dropdown(options=alignment_options, value="rs", description="Alinhamento:", style=style)
        self.widgets['timer_x_widget'] = widgets.IntSlider(value=self.video_props['w'] - 20, min=0, max=self.video_props['w'], description="Posição X:", style=style)
        self.widgets['timer_y_widget'] = widgets.IntSlider(value=self.video_props['h'] - 20, min=0, max=self.video_props['h'], description="Posição Y:", style=style)
        self.widgets['timer_outline_enable'] = widgets.Checkbox(value=True, description="Habilitar Contorno", indent=False)
        self.widgets['timer_outline_width'] = widgets.IntSlider(value=3, min=1, max=20, description="Largura Contorno:", style=style)
        self.widgets['timer_outline_color'] = widgets.ColorPicker(concise=True, description="Cor Contorno:", value="#000000", style=style)
        outline_box = widgets.HBox([self.widgets['timer_outline_width'], self.widgets['timer_outline_color']])
        self.widgets['input_time_format'] = widgets.Dropdown(options=all_time_formats, value="M:SS.ms2", description="Formato de Entrada:", style=style)
        self.widgets['main_timer_format'] = widgets.Dropdown(options=all_time_formats, value="M:SS.ms2", description="Formato no Vídeo:", style=style)
        self.widgets['render_button'] = widgets.Button(description="▶️ RENDERIZAR VÍDEO", button_style='primary', icon='cogs', layout=widgets.Layout(width='98%', height='50px'))
        info_box = widgets.HTML(f"<b>Info:</b> {self.video_props['w']}x{self.video_props['h']} @ {self.video_props['fps']:.2f}fps ({self.video_props['frames']} frames)")
        preview_box = widgets.VBox([self.widgets['preview_out'], self.widgets['preview_slider'], self.widgets['preview_time_label'], self._build_main_navigation_panel()])
        run_config_box = widgets.VBox([self.widgets['input_time_format'], self.widgets['start_run_widget'], self.widgets['start_run_preview'], self.widgets['end_run_widget'], self.widgets['end_run_preview']])
        loads_box = widgets.VBox([self.widgets['add_load_btn'], self.widgets['load_container']])
        customization_box = widgets.VBox([self.widgets['font_widget'], self.widgets['timer_font_size'], self.widgets['timer_color_widget'], self.widgets['timer_alignment'], self.widgets['timer_x_widget'], self.widgets['timer_y_widget'], self.widgets['timer_outline_enable'], outline_box, self.widgets['main_timer_format']])
        accordion = widgets.Accordion(children=[run_config_box, loads_box, customization_box])
        accordion.set_title(0, '1. Configurações da Run'); accordion.set_title(1, '2. Definir Loads'); accordion.set_title(2, '3. Customizar Timer')
        main_container = widgets.VBox([info_box, preview_box, accordion, widgets.HTML("<hr>"), self.widgets['render_button'], self.widgets['render_output']])
        self._setup_observers(); clear_output(wait=True); display(main_container)
        self._on_outline_toggle({'new': self.widgets['timer_outline_enable'].value})
        self._update_all_previews()

    def _on_outline_toggle(self, change):
        is_enabled = change.get('new', False)
        self.widgets['timer_outline_width'].disabled = not is_enabled; self.widgets['timer_outline_color'].disabled = not is_enabled
        self._update_all_previews()

    def _setup_observers(self):
        for w_name in ['font_widget', 'timer_font_size', 'timer_color_widget', 'timer_x_widget', 'timer_y_widget', 'main_timer_format', 'timer_alignment', 'timer_outline_width', 'timer_outline_color']:
            self.widgets[w_name].observe(self._update_all_previews, 'value')
        self.widgets['preview_slider'].observe(lambda c: self._show_frame_with_overlay(c['new'], self.widgets['preview_out']), 'value')
        self.widgets['start_run_widget'].observe(lambda c: self._show_frame_with_overlay(self._parse_time_to_frames(c['new'], self.video_props['fps']), self.widgets['start_run_preview'], is_sub=True), 'value')
        self.widgets['end_run_widget'].observe(lambda c: self._show_frame_with_overlay(self._parse_time_to_frames(c['new'], self.video_props['fps']), self.widgets['end_run_preview'], is_sub=True), 'value')
        self.widgets['input_time_format'].observe(self._on_input_format_change, 'value')
        self.widgets['add_load_btn'].on_click(self._add_load_entry)
        self.widgets['render_button'].on_click(self._on_render_button_clicked)
        self.widgets['timer_outline_enable'].observe(self._on_outline_toggle, 'value')

    def _add_load_entry(self, b=None):
        idx = len(self.load_entries)
        default_time = "0:00.00" if not self.load_entries else self.load_entries[-1]['end_widget'].value
        format_str = self.widgets['input_time_format'].value
        start_widget = widgets.Text(value=self._format_time(self._parse_time_to_seconds(default_time), format_str), description="Início:", style={'description_width': 'initial'})
        end_widget = widgets.Text(value=self._format_time(self._parse_time_to_seconds(default_time), format_str), description="Fim:", style={'description_width': 'initial'})
        remove_btn = widgets.Button(icon='trash', button_style='danger', layout=widgets.Layout(width='auto'))
        title_label = widgets.Label(f"Load #{idx+1}")
        set_start_btn = widgets.Button(description="Set", layout=widgets.Layout(width='auto'))
        set_end_btn = widgets.Button(description="Set", layout=widgets.Layout(width='auto'))
        def on_set_load_click(target_widget): target_widget.value = self._format_time(self.widgets['preview_slider'].value / self.video_props['fps'], format_str)
        set_start_btn.on_click(lambda b, w=start_widget: on_set_load_click(w)); set_end_btn.on_click(lambda b, w=end_widget: on_set_load_click(w))
        start_preview, end_preview = widgets.Output(), widgets.Output()
        new_item = {'id': idx, 'start_widget': start_widget, 'end_widget': end_widget, 'remove_btn': remove_btn, 'title_label': title_label}
        new_item['ui'] = widgets.VBox([widgets.HBox([title_label, remove_btn]), widgets.HBox([start_widget, set_start_btn]), start_preview, widgets.HBox([end_widget, set_end_btn]), end_preview], layout=widgets.Layout(border='1px solid lightgrey', margin='10px 0', padding='8px'))
        start_widget.observe(lambda c, out=start_preview: self._show_frame_with_overlay(self._parse_time_to_frames(c['new'], self.video_props['fps']), out, is_sub=True), 'value')
        end_widget.observe(lambda c, out=end_preview: self._show_frame_with_overlay(self._parse_time_to_frames(c['new'], self.video_props['fps']), out, is_sub=True), 'value')
        self.load_entries.append(new_item); self._redraw_loads_ui()
        self._show_frame_with_overlay(self._parse_time_to_frames(default_time, self.video_props['fps']), start_preview, is_sub=True)
        self._show_frame_with_overlay(self._parse_time_to_frames(default_time, self.video_props['fps']), end_preview, is_sub=True)

    def _redraw_loads_ui(self):
        for i, item in enumerate(self.load_entries):
            item['title_label'].value = f"Load #{i+1}"
            def create_remove_handler(idx):
                def on_remove(b): self.load_entries.pop(idx); self._redraw_loads_ui(); self._update_all_previews()
                return on_remove
            item['remove_btn'].on_click(create_remove_handler(i), remove=True)
            item['remove_btn'].on_click(create_remove_handler(i))
        self.widgets['load_container'].children = tuple(item['ui'] for item in self.load_entries)

    def _get_current_config(self):
        return {
            "timer_x": self.widgets['timer_x_widget'].value, "timer_y": self.widgets['timer_y_widget'].value, "timer_color": webcolors.hex_to_rgb(self.widgets['timer_color_widget'].value),
            "start_run_frame": self._parse_time_to_frames(self.widgets['start_run_widget'].value, self.video_props['fps']), "end_run_frame": self._parse_time_to_frames(self.widgets['end_run_widget'].value, self.video_props['fps']),
            "load_frames": [(self._parse_time_to_frames(e['start_widget'].value, self.video_props['fps']), self._parse_time_to_frames(e['end_widget'].value, self.video_props['fps'])) for e in self.load_entries],
            "t_font_path": self.font_map[self.widgets['font_widget'].value], "t_font_size": int(self.widgets['timer_font_size'].value), "main_timer_format": self.widgets['main_timer_format'].value,
            "timer_alignment": self.widgets['timer_alignment'].value, "timer_outline_enable": self.widgets['timer_outline_enable'].value,
            "timer_outline_width": self.widgets['timer_outline_width'].value, "timer_outline_color": webcolors.hex_to_rgb(self.widgets['timer_outline_color'].value),
        }

    def _draw_timer_on_image(self, image, frame_number, config, scale_factor=1.0):
        draw = ImageDraw.Draw(image)
        ingame_time = self._calculate_ingame_seconds(frame_number, config['start_run_frame'], config['load_frames'], self.video_props['fps'])
        final_run_time = self._calculate_ingame_seconds(config['end_run_frame'], config['start_run_frame'], config['load_frames'], self.video_props['fps'])
        if final_run_time > 0 and ingame_time >= final_run_time: ingame_time = final_run_time
        timer_str = self._format_time(ingame_time, config['main_timer_format'])
        pos = (int(config['timer_x'] * scale_factor), int(config['timer_y'] * scale_factor)); font = ImageFont.truetype(config['t_font_path'], max(8, int(config['t_font_size'] * scale_factor))); anchor = config['timer_alignment']
        if config['timer_outline_enable']:
            outline_color = config['timer_outline_color']; stroke_width = int(config['timer_outline_width'] * scale_factor)
            for x_offset in range(-stroke_width, stroke_width + 1):
                for y_offset in range(-stroke_width, stroke_width + 1):
                    if x_offset != 0 or y_offset != 0: draw.text((pos[0] + x_offset, pos[1] + y_offset), timer_str, font=font, fill=outline_color, anchor=anchor)
        draw.text(pos, timer_str, font=font, fill=config['timer_color'], anchor=anchor)

    def _show_frame_with_overlay(self, frame_number, out_widget, is_sub=False):
        if self.is_rendering or (self.is_playing and out_widget != self.widgets['preview_out']): return
        self.video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = self.video_capture.read()
        with out_widget:
            clear_output(wait=True)
            if not ret: display(HTML("<p style='color:red;'>Erro ao ler frame.</p>")); return
            video_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)); config = self._get_current_config()
            scale = 1.0; final_img = video_img.copy()
            if is_sub:
                max_width = 320; scale = max_width / video_img.width
                final_img = video_img.resize((max_width, int(video_img.height * scale)), Image.Resampling.LANCZOS)
            self._draw_timer_on_image(final_img, frame_number, config, scale)
            with BytesIO() as buffer:
                final_img.convert("RGB").save(buffer, format="PNG"); img_str = base64.b64encode(buffer.getvalue()).decode()
            display(HTML(f'<img src="data:image/png;base64,{img_str}" style="max-width: 100%; height: auto; border: 1px solid grey;" />'))
        if out_widget == self.widgets['preview_out']:
            self.widgets['preview_time_label'].value = f"Tempo no Vídeo: {self._format_time(frame_number / self.video_props['fps'], self.widgets['input_time_format'].value)} | Frame: {frame_number}"

    def _update_all_previews(self, change=None):
        if self.widgets.get('preview_slider'): self._show_frame_with_overlay(self.widgets['preview_slider'].value, self.widgets['preview_out'])

    def _on_input_format_change(self, change):
        new_format = change['new']
        for w_name in ['start_run_widget', 'end_run_widget']: self.widgets[w_name].value = self._format_time(self._parse_time_to_seconds(self.widgets[w_name].value), new_format)
        for item in self.load_entries:
            for w_name in ['start_widget', 'end_widget']: item[w_name].value = self._format_time(self._parse_time_to_seconds(item[w_name].value), new_format)
        self._update_all_previews()

    def _toggle_ui_enabled(self, enabled=True):
        self.is_rendering = not enabled
        for name, w in self.widgets.items():
            if 'output' not in name and hasattr(w, 'disabled'):
                if name in ['timer_outline_width', 'timer_outline_color'] and not self.widgets['timer_outline_enable'].value: continue
                w.disabled = not enabled
        if 'render_button' in self.widgets: self.widgets['render_button'].disabled = not enabled; self.widgets['render_button'].description = "▶️ RENDERIZAR VÍDEO" if enabled else "Renderizando..."

    def _on_render_button_clicked(self, b):
        if self.is_playing: self._toggle_playback()
        video_only_output, final_output = "video_sem_som.mp4", "video_final.mp4"
        if os.path.exists(video_only_output): os.remove(video_only_output)
        if os.path.exists(final_output): os.remove(final_output)
        self._toggle_ui_enabled(False)
        with self.widgets['render_output']:
            clear_output(wait=True); print("🚀 Iniciando a renderização...")
            try:
                config = self._get_current_config()
                print("\n[ETAPA 1/2] Renderizando os frames do vídeo com o timer...")
                cap = cv2.VideoCapture(self.input_video_path); fourcc = cv2.VideoWriter_fourcc(*'mp4v'); out = cv2.VideoWriter(video_only_output, fourcc, self.video_props['fps'], (self.video_props['w'], self.video_props['h']))
                if not out.isOpened(): raise IOError("ERRO: Não foi possível criar o arquivo de vídeo de saída com OpenCV.")
                with tqdm(total=self.video_props['frames'], desc="Renderizando Frames") as pbar:
                    for frame_num in range(self.video_props['frames']):
                        ret, frame = cap.read()
                        if not ret: break
                        img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                        self._draw_timer_on_image(img_pil, frame_num, config)
                        out.write(cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)); pbar.update(1)
                cap.release(); out.release(); print("✅ Frames renderizados com sucesso.")
                print("\n[ETAPA 2/2] Combinando vídeo com áudio original usando FFmpeg...")
                has_audio_cmd = ['ffprobe', '-v', 'error', '-select_streams', 'a', '-show_entries', 'stream=codec_type', '-of', 'json', self.input_video_path]
                try: result = subprocess.run(has_audio_cmd, capture_output=True, text=True, check=True); has_audio = bool(json.loads(result.stdout)['streams'])
                except Exception: has_audio = True; print("AVISO: Não foi possível verificar a faixa de áudio. Tentando combinar mesmo assim.")
                command = ['ffmpeg', '-y', '-i', video_only_output]
                if has_audio: command.extend(['-i', self.input_video_path])
                if has_audio: command.extend(['-map', '0:v:0', '-map', '1:a:0?', '-c:v', 'copy', '-c:a', 'aac', '-shortest'])
                else: command.extend(['-c:v', 'copy'])
                command.append(final_output)
                result = subprocess.run(command, capture_output=True, text=True)
                if result.returncode != 0: raise subprocess.CalledProcessError(result.returncode, " ".join(result.args), stderr=result.stderr)
                clear_output(wait=True); print(f"✅ Renderização concluída! O arquivo '{os.path.basename(final_output)}' está pronto.")
                print("\n➡️ Para baixar, encontre o arquivo 'video_final.mp4' na lista de arquivos à esquerda, clique com o botão direito (ou toque e segure) e escolha 'Download'.")
            except Exception as e:
                clear_output(wait=True); print("❌ Ocorreu um erro durante a renderização.")
                error_message = str(e.stderr if hasattr(e, 'stderr') else e); display(HTML(f"<pre style='background-color:#ffebee;color:#b71c1c;padding:10px;border-radius:5px;white-space:pre-wrap;'>{error_message}</pre>"))
                if os.path.exists(video_only_output) and os.path.getsize(video_only_output) > 0: print("\nTentando baixar o vídeo SEM SOM que foi gerado..."); # Não podemos mais forçar download
            finally:
                if os.path.exists(video_only_output): os.remove(video_only_output)
                print("\n🧹 Arquivo temporário removido."); self._toggle_ui_enabled(True)

# Inicia a aplicação
app = VideoTimerApp()
